### Using ticdat to convert data between different schemas
Here I'm showing how I convert  the testing data when the `input_schema` changes between versions of the package under development (i.e. `tts_netflow_a`).

This notebook needs `tts_netflow_a` in the Python path, and `test_tts_netflow_a` as a subdirectory of the current directory. If you run it directly from the `notebooks` directory cloned from GitHub it won't find `test_tts_netflow_a`. I prefer to use symbolic links for such things.

In [1]:
import tts_netflow_a
import os
from ticdat import TicDatFactory

Only one file to convert here, but I'll demonstrate in a way that extends to more than one.

In [2]:
files =  os.listdir(os.path.join("test_tts_netflow_a", "data"))
files

['netflow_flows_figure_5.json']

This notebook is being run when `tts_netflow_a.input_schema` reflects the new schema, but the data in `files` represents the old schema. To convert, I will thus need the old schema. This is easy to manually copy from the [0.0.1](https://github.com/ticdat/tts_netflow_a/blob/0.0.1/tts_netflow_a/netflow.py#L12) tag.

In [3]:
old_input_schema = TicDatFactory (
    nodes=[["Name"], ["Inflow"]],
    arcs= [["Source", "Destination"], ["Cost", "Capacity"]],
)

In [4]:
for f in files:
    dat = old_input_schema.json.create_tic_dat(os.path.join("test_tts_netflow_a", "data", f))
    # I recommend two monitors so as to study both schemas at the same time
    new_dat = tts_netflow_a.input_schema.TicDat(
        nodes=[[k] for k in dat.nodes], 
        commodities = [["p", 1.0]], 
        arcs = [[s, d, r["Capacity"]] for (s, d), r in dat.arcs.items()], 
        cost = [["p", s, d, r["Cost"]] for (s, d), r in dat.arcs.items()], 
        inflow = [["p", i, r["Inflow"]] for i, r in dat.nodes.items()])
    # make sure its kosher before you write it back
    assert not tts_netflow_a.input_schema.find_foreign_key_failures(new_dat)
    assert not tts_netflow_a.input_schema.find_data_type_failures(new_dat)
    tts_netflow_a.input_schema.json.write_file(new_dat, 
                                               os.path.join("test_tts_netflow_a", "data", f), 
                                               allow_overwrite=True)
    